## Problem Statement
Say that a factory produces four different products, and that the daily produced amount of the first product is x₁, the amount produced of the second product is x₂, and so on. The goal is to determine the profit-maximizing daily production amount for each product, bearing in mind the following conditions:

The profit per unit of product is $20, $12, $40, and $25 for the first, second, third, and fourth product, respectively.

Due to manpower constraints, the total number of units produced per day can't exceed fifty.

For each unit of the first product, three units of the raw material A are consumed. Each unit of the second product requires two units of the raw material A and one unit of the raw material B. Each unit of the third product needs one unit of A and two units of B. Finally, each unit of the fourth product requires three units of B.

Due to the transportation and storage constraints, the factory can consume up to one hundred units of the raw material A and ninety units of B per day.


## Solution
**Decision Variables** = a, b, c, d<br>
**Objective Function**- Maximize Profit = 20a + 12b + 40c + 25d<br>
**Constraints**-<br>
1. sum of all products/manpower = a+b+c+d <= 50
2. raw material A = 3a+2b+c <= 100
3. raw material B = b+2c+3d <= 90
4. a, b, c, d >=0


Installation - <br>
1. pip install scipy pulp
2. on linux- run command "pulptest" in cmd
3. on windows download- https://www.gnu.org/software/glpk/#downloading
4. install glpk- conda install -c conda-forge glpk or sudo apt-get install glpk-utils
5. check version- glpsol --version

## Using Scipy
* SciPy can’t work with integer decision variables.
* SciPy can’t run various external solvers.
* SciPy doesn’t allow you to define maximization problems directly. You must convert them to minimization problems.
* SciPy doesn’t allow you to define constraints using the greater-than-or-equal-to sign directly. You must use the less-than-or-equal-to instead.

In [8]:
from scipy.optimize import linprog
# linprog() solves only minimization (not maximization) problems
# doesn't allow inequality constraints with the greater than or equal to sign (≥), 
# hence modify problem accordingly convert maximize to minimize and greater than equal to less than equal

# maximize doesn't work has changing the sign to make minimize problem
obj = [-20, -12, -40, -25]
#       ─┬   ─┬
#        │    └┤ Coefficient for b
#        └─────┤ Coefficient for a

# inequality constraint left side
lhs_ineq = [[1, 1, 1, 1],
            [3, 2, 1, 0],
            [0, 1, 2, 3]]  

# inequality constraint right side
rhs_ineq = [50,  
            100,  
             90]  

# equality constraint left side
# lhs_eq = [[-1, 5]]  
# equality constraint right side
# rhs_eq = [15]       

# x & y constraints
bnd = [(0, float("inf")),  # Bounds of a
       (0, float("inf")),  # Bounds of b
       (0, float("inf")),
       (0, float("inf"))  
       ]  

In [9]:
opt = linprog(c=obj, 
              A_ub=lhs_ineq, 
              b_ub=rhs_ineq,
            #   A_eq=lhs_eq, 
            #   b_eq=rhs_eq, 
              bounds=bnd,
              method="highs")
opt

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -1900.0
              x: [ 5.000e+00  0.000e+00  4.500e+01  0.000e+00]
            nit: 4
          lower:  residual: [ 5.000e+00  0.000e+00  4.500e+01  0.000e+00]
                 marginals: [ 0.000e+00  1.800e+01  0.000e+00  2.500e+01]
          upper:  residual: [       inf        inf        inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  4.000e+01  0.000e+00]
                 marginals: [-2.000e+01 -0.000e+00 -1.000e+01]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

fun- optimum value of objective<br>
nit- number of iterations need to finish the calculation<br>
x- optimum values of decision variables

The result tells you that the maximal profit is 1900 and corresponds to a = 5 and c = 45. It's not profitable to produce the second and fourth products under the given conditions.
Conclusion-
1. The third product brings the largest profit per unit, so the factory will produce it the most.
2. The first ineqlin: residual is 0, which means that the values of the left and right sides of the manpower (first) constraint are the same. The factory produces 50 units per day, and that's its full capacity.
3. The second ineqlin: residual is 40 because the factory consumes 60 units of raw material A (15 units for the first product plus 45 for the third) out of a potential 100 units.
4. The third ineqlin: residual is 0, which means that the factory consumes all 90 units of the raw material B. This entire amount is consumed for the third product. That's why the factory can't produce the second or fourth product at all and can't produce more than 45 units of the third product. It lacks the raw material B.

# Using PuLP
You can also use PuLP to solve mixed-integer linear programming problems. To define an integer or binary variable, just pass cat="Integer" or cat="Binary" to LpVariable. 

In [10]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [19]:
# Create the model
model = LpProblem(name="resource-allocation", sense=LpMaximize)
# use the sense parameter to choose whether to perform minimization 
# (LpMinimize or 1, which is the default) or maximization (LpMaximize or -1)

# Initialize the decision variables
a = LpVariable(name="a", lowBound=0)
b = LpVariable(name="b", lowBound=0)
c = LpVariable(name="c", lowBound=0)
d = LpVariable(name="d", lowBound=0)
# or use x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 5)} and use x[1], x[2] and so on as variable names

# Add the constraints to the model
model += (a + b + c + d <= 50, "man power")
model += (3*a + 2*b + 1*c <= 100, "raw material A")
model += (b + 2*c + 3*d <= 90, "raw material B")

# Add the objective function to the model
model += 20*a + 12*b + 40*c + 25*d # or use lpSum([20*a, 12*b, 40*c, 25*d])
model


resource-allocation:
MAXIMIZE
20*a + 12*b + 40*c + 25*d + 0
SUBJECT TO
man_power: a + b + c + d <= 50

raw_material_A: 3 a + 2 b + c <= 100

raw_material_B: b + 2 c + 3 d <= 90

VARIABLES
a Continuous
b Continuous
c Continuous
d Continuous

In [20]:
# Solve the problem
status = model.solve() # argument for different solver- solver=GLPK(msg=False) but also use- from pulp import GLPK

print(f"status: {model.status}, {LpStatus[model.status]}")

print(f"objective: {model.objective.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /var/tmp/7c1d6bf548434852ad90f82a3a31f153-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/tmp/7c1d6bf548434852ad90f82a3a31f153-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 23 RHS
At line 27 BOUNDS
At line 28 ENDATA
Problem MODEL has 3 rows, 4 columns and 10 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (0) rows, 4 (0) columns and 10 (0) elements
0  Obj -0 Dual inf 97 (4)
0  Obj -0 Dual inf 97 (4)
2  Obj 1900
Optimal - objective value 1900
Optimal objective 1900 - 2 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

status: 1, Optimal
objective: 1900.0
a: 5.0
b: 0.0
c: 45.0
d: 0.0
man_power: 0.

Let’s make this problem more complicated and interesting. Say the factory can’t produce the first and third products in parallel due to a machinery issue. What’s the most profitable solution in this case?

Now you have another logical constraint: if a is positive, then c must be zero and vice versa. This is where binary decision variables are very useful. You’ll use two binary decision variables, y₁ and y₃, that’ll denote if the first or third products are generated at all:

In [25]:
model = LpProblem(name="resource-allocation", sense=LpMaximize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 5)}
y = {i: LpVariable(name=f"y{i}", cat="Binary") for i in (1, 3)}

# Add constraints
model += (lpSum(x.values()) <= 50, "manpower")
model += (3*x[1] + 2*x[2] + x[3] <= 100, "material_a")
model += (x[2] + 2*x[3] + 3*x[4] <= 90, "material_b")

M = 100 # defines an arbitrarily large number M. The value 100 is large enough in this case because you can’t have more than 100 units per day.
model += (x[1] <= y[1]*M, "a_constraint") # if y[1] is zero, then x[1] must be zero, if y[1] is 1 then x[1] can have maximum value of M.
model += (x[3] <= y[3]*M, "c_constraint") #  if y[3] is zero, then x[3] must be zero, if y[3] is 1 then x[3] can have maximum value of M.
model += (y[1] + y[3] <= 1, "y_constraint") # either y[1] or y[3] is zero (or both are), so either x[1] or x[3] must be zero as well.

# Set objective
model += 20*x[1] + 12*x[2] + 40*x[3] + 25*x[4]

# Solve the optimization problem
status = model.solve()

print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in model.variables():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /var/tmp/015c4c8564e0459496e29594f5f95594-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/tmp/015c4c8564e0459496e29594f5f95594-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 36 RHS
At line 43 BOUNDS
At line 46 ENDATA
Problem MODEL has 6 rows, 6 columns and 16 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1900 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 6 rows, 6 columns (2 integer (2 of which binary)) and 16 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 4.44089e-16
Cbc0038I Solution found of -1800
Cbc0038I Relaxing continuous gives -1800
Cbc0038I Before mini branch and bound, 2 in